# 11월 27일 토 공부

# 04. 머신러닝 핵심 알고리즘

## 1. 로지스틱 회귀 모델

In [1]:
# 분석 데이터 준비
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
data = pd.read_csv('/Users/yerim/Desktop/빅데이터분석기사/BigData_data_최종본(1022)/breast-cancer-wisconsin.csv', encoding='utf-8')
X = data[data.columns[1:10]]
y = data[["Class"]]

In [2]:
# 데이터 분류
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=42)

In [3]:
# 데이터 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_minmax_train = scaler.transform(X_train)
X_scaled_minmax_test = scaler.transform(X_test)

In [4]:
# 기본 모델 적용 - 로지스틱회귀모델
# 하이퍼파라미터 C = 1
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_scaled_minmax_train, y_train)

pred_train = model.predict(X_scaled_minmax_train)
pred_test = model.predict(X_scaled_minmax_test)

print("훈련데이터 점수(기본 모델 적용, C=1) : ", model.score(X_scaled_minmax_train, y_train))
print("테스트데이터 점수(기본 모델 적용, C=1) : ", model.score(X_scaled_minmax_test, y_test))

훈련데이터 점수(기본 모델 적용, C=1) :  0.97265625
테스트데이터 점수(기본 모델 적용, C=1) :  0.9590643274853801


In [5]:
# 기본 모델 성능 분석 - 혼동행렬
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
confusion_test = confusion_matrix(y_test, pred_test)

print("훈련 데이터 혼동행렬(기본 모델 적용, C=1) : \n" , confusion_train )
print("테스트 데이터 혼동행렬(기본 모델 적용, C=1) : \n", confusion_test )

훈련 데이터 혼동행렬(기본 모델 적용, C=1) : 
 [[328   5]
 [  9 170]]
테스트 데이터 혼동행렬(기본 모델 적용, C=1) : 
 [[106   5]
 [  2  58]]


In [6]:
# 기본 모델 성능 분석 - 평가 지표
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
cfreport_test = classification_report(y_test, pred_test)

print("훈련 데이터 상세 평가 지표 (기본 모델 적용, C=1) : \n" , cfreport_train )
print("테스트 데이터 상세 평가 지표 (기본 모델 적용, C=1) : \n" , cfreport_test )

훈련 데이터 상세 평가 지표 (기본 모델 적용, C=1) : 
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       333
           1       0.97      0.95      0.96       179

    accuracy                           0.97       512
   macro avg       0.97      0.97      0.97       512
weighted avg       0.97      0.97      0.97       512

테스트 데이터 상세 평가 지표 (기본 모델 적용, C=1) : 
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



In [7]:
# 파라미터튜닝 - Grid Search
from sklearn.model_selection import GridSearchCV
parameters = {'C' : [0.001, 0.01, 0.1 , 1.0 , 10, 100]}
grid_search = GridSearchCV(model, parameters , cv=5)
grid_search.fit(X_scaled_minmax_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1.0, 10, 100]})

In [8]:
# GridSearch결과 확인
print("최적의 파라미터 : {}".format(grid_search.best_params_))
print("최적 훈련데이터 성능 : {:.4f}".format(grid_search.best_score_))
print("Test data Score : {:.4f}".format(grid_search.score(X_scaled_minmax_test, y_test)))

최적의 파라미터 : {'C': 10}
최적 훈련데이터 성능 : 0.9726
Test data Score : 0.9591


In [12]:
# 파라미터 튜닝- random search
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
parameters = {'C' : randint(low=0.001,high=100)}
random_search = RandomizedSearchCV(model, param_distributions=parameters, cv=5, n_iter=100)
random_search.fit(X_scaled_minmax_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fde334388b0>})

In [13]:
# RandomSearch 결과 확인
print("최적의 파라미터 : {}".format(random_search.best_params_))
print("최적의 스코어 : {:.4f}".format(random_search.best_score_))
print("Test data Score : {:.4f}".format(model.score(X_scaled_minmax_test, y_test)))

최적의 파라미터 : {'C': 12}
최적의 스코어 : 0.9745
Test data Score : 0.9591


## 2. K 최근접 이웃법

### 2-1 ) 분류 (Classification)

In [16]:
# 분석 데이터 준비
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
data = pd.read_csv('/Users/yerim/Desktop/빅데이터분석기사/BigData_data_최종본(1022)/breast-cancer-wisconsin.csv', encoding='utf-8')
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   code                         683 non-null    int64
 1   Clump_Thickness              683 non-null    int64
 2   Cell_Size                    683 non-null    int64
 3   Cell_Shape                   683 non-null    int64
 4   Marginal_Adhesion            683 non-null    int64
 5   Single_Epithelial_Cell_Size  683 non-null    int64
 6   Bare_Nuclei                  683 non-null    int64
 7   Bland_Chromatin              683 non-null    int64
 8   Normal_Nucleoli              683 non-null    int64
 9   Mitoses                      683 non-null    int64
 10  Class                        683 non-null    int64
dtypes: int64(11)
memory usage: 58.8 KB


In [17]:
# X 와 y 나누기
X = data[data.columns[1:10]]
y = data[['Class']]

In [18]:
# train-test-split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=42)

In [19]:
# 정규화 하기 - minmax 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

In [21]:
# 모델 사용하기 - KNN 알고리즘(분류)
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_scaled_train, y_train)

pred_train = model.predict(X_scaled_train)
print("훈련데이터 성능 : " , model.score(X_scaled_train, y_train))

pred_test = model.predict(X_scaled_test)
print("테스트데이터 성능 : ", model.score(X_scaled_test, y_test))

훈련데이터 성능 :  0.984375
테스트데이터 성능 :  0.9532163742690059


In [23]:
# 기본 모델의 혼동행렬 확인하기
from sklearn.metrics import confusion_matrix

confusion_train = confusion_matrix(y_train, pred_train)
print("훈련 데이터 혼동행렬 : \n" , confusion_train)

confusion_test = confusion_matrix(y_test, pred_test)
print("테스트 데이터 혼동행렬 : \n" , confusion_test)

훈련 데이터 혼동행렬 : 
 [[331   2]
 [  6 173]]
테스트 데이터 혼동행렬 : 
 [[106   5]
 [  3  57]]


In [25]:
# 기본 모델의 상세 지표 확인하기
from sklearn.metrics import classification_report

cfreport_train = classification_report(y_train, pred_train)
print("훈련 데이터 상세 지표 : \n ", cfreport_train)

cfreprot_test = classification_report(y_test, pred_test)
print("테스트 데이터 상세 지표 : \n", cfreport_test)

훈련 데이터 상세 지표 : 
                precision    recall  f1-score   support

           0       0.98      0.99      0.99       333
           1       0.99      0.97      0.98       179

    accuracy                           0.98       512
   macro avg       0.99      0.98      0.98       512
weighted avg       0.98      0.98      0.98       512

테스트 데이터 상세 지표 : 
               precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



In [26]:
# 파라미터 튜닝 - Grid Search
from sklearn.model_selection import GridSearchCV
parameters = {'n_neighbors' : [1, 3, 5, 7, 9, 11]}
grid_search = GridSearchCV(model, parameters, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]})

In [28]:
# Grid Search 결과 보기
print("훈련 데이터 가장 좋은 파라미터 : " , grid_search.best_params_)
print("훈련 데이터 가장 높은 성능 : {:.4f}".format(grid_search.best_score_))
print("테스트 데이터 점수 : {:.4f}".format(model.score(X_scaled_test, y_test)))

훈련 데이터 가장 좋은 파라미터 :  {'n_neighbors': 3}
훈련 데이터 가장 높은 성능 : 0.9824
테스트 데이터 점수 : 0.9532


In [31]:
# 파라미터 튜닝 - Random Search
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
parameters = {'n_neighbors' : randint(low=1, high=20)}
random_search = RandomizedSearchCV(model, param_distributions=parameters, cv=5, n_iter=20)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=20,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fde33553bb0>})

In [34]:
# Random Search 결과 보기
print("훈련 데이터 가장 좋은 파라미터 : ", random_search.best_params_)
print("훈련 데이터 가장 좋은 성능 : {:.4f}".format(random_search.best_score_))
print("테스트 데이터 성능 : {:.4f}".format(model.score(X_scaled_test, y_test)))

훈련 데이터 가장 좋은 파라미터 :  {'n_neighbors': 4}
훈련 데이터 가장 좋은 성능 : 0.9804
테스트 데이터 성능 : 0.9532


### 2-2) 회귀(Regression)

In [35]:
# 분석 데이터 준비
import pandas as pd
data = pd.read_csv('/Users/yerim/Desktop/빅데이터분석기사/BigData_data_최종본(1022)/house_price.csv', encoding='utf-8')
X=data[data.columns[1:5]]
y=data[["house_value"]]

In [36]:
# train-test split
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [ ]:
# MinMax정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)